# Описание проекта

Мы будем работать над чат-ботом.  
Когда мы слышим слово «чат-бот», в голове возникают ассоциации с генеративными нейронными сетями, сложнейшими алгоритмами и тому подобные вещами, но на самом деле всё гораздо проще, во многом из-за бизнес-ограничений:
- Модель должна быть предсказуемой. Представьте, что чат-бот начнёт путать времена работы ваших отделений. Генеративная сеть не даст вам гарантии правильного ответа.
- Модель должна быть корректной: не ругаться матом, воздерживаться от сексистских, расистских высказываний.

Поэтому зачастую на конференциях говорят о продвинутых технологиях, но в жизни используют KNN по заготовленным ответам.

### Установка библиотек

In [2]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=13890c96a3b682153c889d4ff1e6a4562d30a15fd7a274ee4787ac07b9b07b32
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [3]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c892740b78bd99a50a1b3912f01770e44686dc18fb10047a968ff04b65964e8c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [4]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=b0eea6d1e64fa22885ea76572ebd991521a45f1a453fcacab759c954e314110e
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


### Загрузка библиотек

In [5]:
import os
import string
import annoy
import codecs
import numpy as np
import pandas as pd
import warnings
import pickle

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

warnings.filterwarnings('ignore')

### Создадание простого чат-бота

Предобработаем ответы mail.ru из файла: к каждому вопросу присоединим 1 ответ и запишем в файл на будущее. Это позволит нам сэкономить время и ресурсы при дальнейшем препроцессинге текста.

In [ ]:
question = None
written = False

# мы идём по всем записям разделённым знаком ---
# объединяем первую и вторую строки как вопрос(\t)ответ
with codecs.open(
    '/content/drive/MyDrive/prepared_answers.txt',
    'w',
    'utf-8'
) as fout:
    with codecs.open('/content/drive/MyDrive/Otvety.txt', 'r', 'utf-8') as fin:
        for line in tqdm(fin):
            if line.startswith('---'):
                written = False
                continue
            if not written and question is not None:
                fout.write(
                    question
                    .replace('\t', ' ')
                    .strip()
                    + '\t'
                    + line.replace('\t', ' ')
                )
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

Теперь нам нужно предобработать текст, чтобы обучить word2vec и получить эмбеддинги. Удаляем знаки препинания и делаем лемматизацию.

In [6]:
def preprocess_txt(line):
    spls = ''.join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != '']
    return spls

In [ ]:
sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))
exclude = set(string.punctuation)
c = 0

with codecs.open('/content/drive/MyDrive/Otvety.txt', 'r', 'utf-8') as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

0it [00:00, ?it/s]

In [ ]:
# обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(
    sentences = sentences,
    vector_size = 100,
    min_count = 1,
    window = 5
)
model.save('/content/drive/MyDrive/w2v_model')

Теперь нам нужно сложить в индекс все вопросы. Используем библиотеку annoy. Проходимся по всем ответам, считаем, что вектор предложения - сумма word2vecов слов, которые входят в него (конечно же усреднённая).

In [ ]:
index = annoy.AnnoyIndex(100, 'angular')
index_map = {}
counter = 0

with codecs.open(
    '/content/drive/MyDrive/prepared_answers.txt',
    'r',
    'utf-8'
) as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split('\t')
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model.wv:
                vector += model.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
        counter += 1

index.build(10)
index.save('/content/drive/MyDrive/speaker.ann')

0it [00:00, ?it/s]

True

Теперь остаётся реализовать метод, который получит на вход вопрос и найдёт ответ к нему! Мы препроцессим вопрос, находим ближайший вопрос и выбираем ответ на ближайший вопрос.

In [7]:
def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [ ]:
# проверяем работу чата
print(find_answer('Как пройти в библиотеку?'))
print(find_answer('Где провести отпуск?'))
print(find_answer('Где деньги лежат?'))

луче в косерваторию-там консервы. 

На курорт, к теплому ласковому морю, под пальмы :-). 

В кошельке!))). 



### Обучаем классификатор: продуктовый запрос vs. всё остальное

In [12]:
# загружаем продуктовый датафрейм
products = pd.read_csv('/content/drive/MyDrive/ProductsDataset.csv')

# объединяем название и описание в одну строку
products = products.fillna('')
products['text'] = (
    products['title'].str.strip()
    + ' '
    + products['descrirption'].str.strip()
)

# добавляем метку для классификации
products['target'] = 1
products.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text,target
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"Юбка детская ORBY Новая, не носили ни разу. В ...",1
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"Ботильоны Новые,привезены из Чехии ,указан раз...",1
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,Брюки Размер 40-42. Брюки почти новые - не зна...,1
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"Продам детские шапки Продам шапки,кажда 200р.Р...",1
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"Блузка Темно-синяя, 42 размер,состояние отличн...",1


In [ ]:
# для всего остального возьмём такое же количество
# вопросов из файла "prepared_answers.txt"
text = []

with codecs.open(
    '/content/drive/MyDrive/prepared_answers.txt',
    'r',
    'utf-8'
) as f:
    for line in tqdm(f):
        spls = line.split('\t')
        if len(spls[0]) > 2:
          text.append(spls[0])
        if len(text) == len(products):
            break

questions = pd.DataFrame({'text': text, 'target': 0})
questions.head()

0it [00:00, ?it/s]

,text,target
0,Как парни относятся к цветным линзам? Если у д...,0
1,"Что делать, сегодня нашёл 2 миллиона рублей? .",0
2,Эбу в двенашке называется Итэлма что за эбу? .,0
3,академия вампиров. сколько на даный момент час...,0
4,как защититься от энергетического вампира .,0


In [8]:
# создаём вспомогательную функцию
def w2v_converter(text):
    """Функция преобразовывает текст в вектор"""
    # препроцессинг текста
    preprocessed_text = preprocess_txt(text)
    # векторизация текста
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_text:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector

In [ ]:
# объединяем продукты и вопросы в один датафрейм
data = pd.concat([products[['text', 'target']], questions])

# перемешиваем датафрейм для конспирации :)
data = data.sample(frac = 1, random_state = 42).reset_index(drop = True)

# преобразовываем текст в вектор
data['text'] = data['text'].apply(w2v_converter)

# разбиваем значения вектора на отдельные колонки
for i in range(100):
    data[str(i)] = data['text'].apply(lambda x: x[i])

# удаляем колонку с векторами
data = data.drop('text', axis = 1)
data.head()

,target,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,-0.354522,0.314456,0.501296,0.243391,0.272514,-0.740517,-0.158067,0.781221,-0.338800,...,0.411691,-0.141635,-0.062138,-0.426368,0.434774,0.005718,-0.236658,-0.183634,-0.079462,0.302726
1,0,-0.620137,0.533748,1.096688,0.468271,0.220536,-0.824087,0.401009,0.735789,-1.551453,...,1.319913,-0.329909,0.118505,-0.969109,0.960908,-0.110992,0.103441,-0.547767,0.173600,0.151990
2,0,0.123707,0.811320,0.656406,-0.043262,0.717184,-1.310277,-0.187035,1.969920,-0.287500,...,0.891600,0.070920,-0.367162,0.528299,1.268838,-0.184069,1.048089,-0.867405,0.309627,-0.226240
3,1,-0.490737,0.912238,0.389526,0.578445,0.548184,-1.203240,-0.073237,1.402847,-1.099693,...,0.913032,-0.532978,0.192980,-0.768301,1.072713,0.614072,1.022503,-0.752813,0.252363,0.062904
4,1,-0.500870,1.093189,0.452851,0.884068,0.495700,-1.383850,0.309951,1.705419,-1.090458,...,1.165469,-0.923082,0.542315,-1.026151,1.298445,0.372829,1.610677,-0.941717,0.469002,0.328021


In [ ]:
# отделяем целевой признак от остальных
X = data.drop('target', axis = 1)
y = data['target']

# разделяем выборку на обучающую и валидационную
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify = y, random_state = 42
)

# обучаем логистическую регрессию
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# рассчитываем и выводим метрики на валидации
print('Accuracy: ', round(metrics.accuracy_score(y_test, y_pred), 4))
print('F1:       ', round(metrics.f1_score(y_test, y_pred), 4))
print('Precision:', round(metrics.precision_score(y_test, y_pred), 4))
print('Recall:   ', round(metrics.recall_score(y_test, y_pred), 4))

Accuracy:  0.9412
F1:        0.9413
Precision: 0.9401
Recall:    0.9425


In [9]:
# реализуем функцию определяющую класс запроса
def get_class(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return lr.predict(vector.reshape(1, -1))[0]

In [21]:
# проверяем работу классификатора
print(
    'Запрос: Хочу платье с цветочным орнаментом.\nКласс:',
    get_class('Хочу платье с цветочным орнаментом')
)
print()
print(
    'Запрос: Когда будет дождь?\nКласс:',
    get_class('Когда будет дождь?')
)

Запрос: Хочу платье с цветочным орнаментом.
Класс: 1

Запрос: Когда будет дождь?
Класс: 0


In [ ]:
# сериализуем модель и записываем результат в файл
with open('/content/drive/MyDrive/get_class_lr.pkl', 'wb') as output:
    pickle.dump(lr, output)

### Добавляем логику поиска похожих товаров по продуктовому запросу

In [ ]:
# Обучим модель word2vec на названиях и описаниях товаров
sentences = []

for text in products['text'].values:
    sentences.append(preprocess_txt(text))

product_model = Word2Vec(
    sentences = sentences,
    vector_size = 100,
    min_count = 1,
    window = 5
)
product_model.save('/content/drive/MyDrive/w2v_products')

In [10]:
# преобразуем вспомогательнуй функцию под продуктовую word2vec модель
def w2v_product_converter(text):
    """Функция преобразовывает текст в вектор"""
    preprocessed_text = preprocess_txt(text)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_text:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector

In [ ]:
# сворачиваем названия и описания товаров в векторное представление Word2Vec
products['title_w2v'] = products['title'].apply(w2v_product_converter)
products['descrirption_w2v'] = products['descrirption'].apply(w2v_product_converter)

In [ ]:
# строим продуктовые индексы и карту с кодами и названиями продуктов
product_index = annoy.AnnoyIndex(100, 'angular')
product_map = {}

for i in products.index:
    product_index.add_item(i, products.iloc[i].title_w2v)
    product_map[i] = f'{products.iloc[i].product_id} {products.iloc[i].title}'

for n, i in enumerate(range(len(products), len(products) * 2)):
    product_index.add_item(i, products.iloc[n].descrirption_w2v)
    product_map[i] = f'{products.iloc[n].product_id} {products.iloc[n].title}'

product_index.build(10)
product_index.save('/content/drive/MyDrive/searcher.ann')

True

In [11]:
# реализуем поиск товарных запросов в товарном индексе
def find_product(request):
    preprocessed_request = preprocess_txt(request)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_request:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = product_index.get_nns_by_vector(vector, 1)
    return product_map[answer_index[0]]

In [14]:
print(find_product('Красная блузка'))
print(find_product('Красивые сандали'))
print(find_product('Тёплый свитер'))

59993f2aa09cd541d16d16c2 Белая блузка
5a82c7b42756ba068c437276 Кеды с красивым принтом
589f01a8cd30221660c17a09 Свитер теплый


### Тестирование

In [1]:
# устанавливаем библиотеки
!pip install annoy
!pip install pymorphy2
!pip install stop_words

In [2]:
# загружаем библиотеки
import os
import string
import annoy
import codecs
import numpy as np
import pandas as pd
import warnings
import pickle

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

warnings.filterwarnings('ignore')

In [3]:
# объявляем вспомогательные функции
def preprocess_txt(line):
    spls = ''.join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != '']
    return spls


def find_answer(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]


def w2v_converter(text):
    """Функция преобразовывает текст в вектор"""
    # препроцессинг текста
    preprocessed_text = preprocess_txt(text)
    # векторизация текста
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_text:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector


def get_class(question):
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return lr.predict(vector.reshape(1, -1))[0]


def w2v_product_converter(text):
    """Функция преобразовывает текст в вектор"""
    preprocessed_text = preprocess_txt(text)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_text:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector


def find_product(request):
    preprocessed_request = preprocess_txt(request)
    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_request:
        if word in product_model.wv:
            vector += product_model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = product_index.get_nns_by_vector(vector, 1)
    return product_map[answer_index[0]]

In [4]:
# восстанавливаем вспомогательные переменные дла предобработки теста
exclude = set(string.punctuation)
morpher = MorphAnalyzer()
sw = set(get_stop_words('ru'))

# загружаем Word2Vec модель и индекс для болталки
model = Word2Vec.load('/content/drive/MyDrive/w2v_model')
index = annoy.AnnoyIndex(100, 'angular')
index.load('/content/drive/MyDrive/speaker.ann')

# загружаем Word2Vec модель и индекс для товарных запросов
product_model = Word2Vec.load('/content/drive/MyDrive/w2v_products')
product_index = annoy.AnnoyIndex(100, 'angular')
product_index.load('/content/drive/MyDrive/searcher.ann')

# загружаем модель классификатора из pkl-файла
with open('/content/drive/MyDrive/get_class_lr.pkl', 'rb') as pkl_file:
    lr = pickle.load(pkl_file)

# восстанавливаем карту индексов для болталки
index_map = {}
counter = 0
with codecs.open(
    '/content/drive/MyDrive/prepared_answers.txt',
    'r',
    'utf-8'
) as f:
    for line in tqdm(f):
        spls = line.split('\t')
        index_map[counter] = spls[1]
        counter += 1

# восстанавливаем карту товарных индексов
products = pd.read_csv('/content/drive/MyDrive/ProductsDataset.csv')
product_map = {}
for i in products.index:
    product_map[i] = f'{products.iloc[i].product_id} {products.iloc[i].title}'
for n, i in enumerate(range(len(products), len(products) * 2)):
    product_map[i] = f'{products.iloc[n].product_id} {products.iloc[n].title}'

0it [00:00, ?it/s]

In [5]:
# заворачивем всю логигу в функцию get_answer()
def get_answer(request):
    if get_class(request):
        return find_product(request)
    else:
        return find_answer(request)

In [6]:
# проверяем на запросах
request = 'Юбка детская ORBY'
print('Запрос:', request)
print('Результат:', get_answer(request))
print()

request = 'Где ключи от танка'
print('Запрос:', request)
print('Результат:\n', get_answer(request))

Запрос: Юбка детская ORBY
Результат: 58e3cfe6132ca50e053f5f82 Юбка детская ORBY

Запрос: Где ключи от танка
Результат:
 В зависимости от того насколько дружишь с техникой . 1-день снять, разобрать . 2-день потратить на поиск запчастей (к счастью на наш автопром они на каждом уг), шлифовок и расточек (если это требуется). 3-день тратится на мойку и подготовку запчастей к сбору агрегата. 4-5 дни потратятся на сборку агрегата и установку его на место, не забыть и о сцеплении и при необходимость поменять, итог - от 10-13 т. р без сцепления и + сцепление 2800-3000 в сборе. (в сервисе 3 дня). 



In [7]:
# проверяем на автотестах
print(get_answer('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82'))
print(not get_answer('Где ключи от танка').startswith('5'))

True
True


### Ссылки на сохранёные файлы

[prepared_answers](https://drive.google.com/file/d/1mB2tvdxzUEgiwoPmHpxJCkDltXzt_loK/view?usp=sharing) - подготовленные ответы на вопросы из файла Otvety.txt  
[index](https://drive.google.com/file/d/1-HHRG4rEiod2eGyJwfspmXsq2-oYdO04/view?usp=sharing) - индекс с вопросами из Otvety.txt  
[model](https://drive.google.com/file/d/1-B_ezAZ1TaZTVz9MQ0PEf8o0GyLzqImf/view?usp=sharing) - word2vec модель обученная на строках Otvety.txt

[lr](https://drive.google.com/file/d/1AotT6iWyXhwW1a2e2C3WSAWV3rMV8oG_/view?usp=sharing) - обученный классификатор «товарный запрос vs. болталка»

[product_index](https://drive.google.com/file/d/1-REa34J_Ei4mLs0HJXUd67dpytovpfkU/view?usp=sharing) - индекс с продуктовыми кодами и названиями  
[product_model](https://drive.google.com/file/d/1-PUoFyfDzE7u2GjQtOk2dYZTfUOIa0Wc/view?usp=sharing) - word2vec модель обученная на продуктовых названиях и описаниях